In [84]:
import pickle
import json

with open(f"rawData/rawdata90", 'rb') as f:
    data = pickle.load(f)

with open(f"rawData/uniqueBS90", 'rb') as f:
    unique_bs = pickle.load(f)

In [85]:
class pc_potential:
    def __init__(self, line, id, parent_id, const_line, both_line, largeV_line, minset):
        self.id = id
        self.constraint = line
        self.constc = const_line
        self.bothc = both_line
        self.largec = largeV_line
        self.potential = set()
        self.parent_id = parent_id
        self.child_list = []
        self.minset = minset

class PathConditionTree:
    def __init__(self, unique_branchset):
        self.state_conditions = []
        self.state_tree = {-1:pc_potential(None, -1, None, None, None, None, False)}
        self.unique_branchset = unique_branchset

    def add_newdata(self, pc, bsidx, const_line, both_line, largeV_line, minset):
        parent_id = -1
        for i, c in enumerate(pc):
            sc = pc[:i+1]
            try:
                scidx = self.state_conditions.index(sc)
                if self.state_tree[scidx].minset != minset[0]:
                    self.state_tree[scidx].minset = minset[0]
            except:
                scidx = len(self.state_conditions)
                self.state_conditions.append(sc)
                self.state_tree[scidx] = pc_potential(c, scidx, parent_id, const_line[i], both_line[i], largeV_line[i], minset[0])
                self.state_tree[parent_id].child_list.append(scidx)
            self.state_tree[scidx].potential |= self.unique_branchset[bsidx]
            parent_id = scidx
            

In [86]:
tree_data = PathConditionTree(unique_bs)
for ktest in data:
    bsidx, pc, constpc, bothpc, lvpc, minset = data[ktest].items()
    tree_data.add_newdata(pc[1], bsidx[1], constpc[1], bothpc[1], lvpc[1], minset[1])

In [87]:
data[ktest]["minset"]

[True]

In [88]:
node_data = {}
minset_count = 0
minset_leaf = 0
unique_minset_pc = set()
for node_id, node_info in tree_data.state_tree.items():
    node_data[node_id] = {}
    node_data[node_id]["constraint"] = node_info.constraint
    node_data[node_id]["const_constraint"] = node_info.constc
    node_data[node_id]["both_constraint"] = node_info.bothc
    node_data[node_id]["large_constraint"] = node_info.largec
    node_data[node_id]["children"] = node_info.child_list
    node_data[node_id]["potential"] = len(node_info.potential)
    node_data[node_id]["minset"] = node_info.minset
    if node_info.minset:
        minset_count += 1
        unique_minset_pc.add(node_info.constraint)
    

In [89]:
print(minset_count, " / ", len(tree_data.state_tree))
print(len(unique_minset_pc))
# 1239  /  5650
# 614

1239  /  5650
614


In [90]:
del node_data[-1]
with open(f"nodesData.json90", 'w') as f:
    json.dump(node_data, f, indent=4)